# Scraping house prices data from Zillow

In [2]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd

In [3]:
# Access zillow website and extract the html infor with BeautifulSoup
header = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}
url = "https://www.zillow.com/san-jose-ca/"
page = requests.get(url, headers=header)
soup = BeautifulSoup(page.content, "html.parser")

In [35]:
# finding the total number of pages of the web
pg_num = soup.find('span',{'class':'Text-c11n-8-10-0__aiai24-0 foGDLH'}).text
last_pg = (pg_num.split()[3])
print(last_pg + " pages total")



20 pages total


In [36]:
#finding tag that contain all the houses information
all_info = soup.find('div', {'id':'grid-search-results'}).find('ul').find_all('article',{'class':'list-card list-card_not-saved'})
print(len(all_info))


40


In [6]:
house_list = []
for page in range(1,int(last_pg)+1,1):
    r = requests.get("https://www.zillow.com/san-jose-ca/{}_p".format(page), headers=header)
    soup = BeautifulSoup(r.content, "html.parser")
    all_info = soup.find('div', {'id':'grid-search-results'}).find('ul').find_all('article',{'class':'list-card list-card_not-saved'})
 
    for item in all_info:
        dic={}

        dic['address'] = item.find('address',{'class':'list-card-addr'}).text.split(',')[0]
        
        dic['locality'] = item.find('address',{'class':'list-card-addr'}).text.split(', ',1)[1]
        
        try:
            dic['price'] = float(item.find('div', {'class':'list-card-price'}).text.replace('$','').replace(',','').replace('+','').replace('Est.','')) # price
        except:
            dic['beds']=None
            
        try:
            dic['beds'] = float(item.find('ul', {'class':'list-card-details'}).find_all('li')[0].text.replace('bds','').replace('bd',''))
        except:
            dic['beds']=None
        
        try:
            dic['bath'] = float(item.find('ul', {'class':'list-card-details'}).find_all('li')[1].text.replace('ba',''))
        except:
            dic['bath']= None
        
        try:    
            dic['area_sqft'] = float(item.find('ul', {'class':'list-card-details'}).find_all('li')[2].text.replace('sqft','').replace(',',''))
        except:
            dic['area_sqft']= None
            
        dic['list_type'] = item.find('div',{'class':'list-card-type'}).text

        house_list.append(dic)

In [7]:
# Turn the dictionary into a dataframe
df = pd.DataFrame(house_list)
df

,address,locality,price,beds,bath,area_sqft,list_type
0,1728 Peregrino Way,"San Jose, CA 95125",2998000.0,5.0,4.0,3737.0,House for sale
1,3157 Tristian Ave,"San Jose, CA 95127",799000.0,3.0,2.0,1115.0,House for sale
2,593 Ezie St,"San Jose, CA 95111",749000.0,4.0,2.0,1113.0,House for sale
3,4610 Capitola Ave,"San Jose, CA 95111",848000.0,3.0,2.0,1170.0,House for sale
4,1774 Wilcox Way,"San Jose, CA 95125",2138000.0,4.0,3.0,2370.0,House for sale
...,...,...,...,...,...,...,...
795,2975 Julio Ave,"San Jose, CA 95124",1249888.0,3.0,2.0,1424.0,House for sale
796,175 W Saint James St UNIT 1401,"San Jose, CA 95110",848000.0,2.0,2.0,1103.0,Condo for sale
797,441 Casselino Dr,"San Jose, CA 95136",695000.0,2.0,2.0,1104.0,Contingent
798,5580 Starcrest Dr,"San Jose, CA 95123",945000.0,3.0,2.0,1354.0,Contingent


# Now we have a dataframe contain houses for sale in San Jose, let do some data cleaning and export the dataframe to an excel file.

In [8]:
# Now let's clean up our data 
# describe give us a general look of the data
# there 10 different list_type, but we only want house for sale 
# we can see that there duplicate house address, 800 house addresses and 664 unique house addresses
df.describe(include='all')

,address,locality,price,beds,bath,area_sqft,list_type
count,800,800,7.960000e+02,774.000000,774.000000,790.000000,800
unique,649,40,NaN,NaN,NaN,NaN,10
top,5382 Entrada Cedros,"San Jose, CA 95125",NaN,NaN,NaN,NaN,House for sale
freq,2,87,NaN,NaN,NaN,NaN,329
mean,NaN,NaN,1.022600e+06,3.065891,2.407623,1645.602532,NaN
std,NaN,NaN,8.772022e+05,1.129196,1.007504,887.579248,NaN
min,NaN,NaN,7.500000e+04,1.000000,1.000000,355.000000,NaN
25%,NaN,NaN,5.937500e+05,2.000000,2.000000,1133.000000,NaN
50%,NaN,NaN,8.990000e+05,3.000000,2.000000,1440.000000,NaN
75%,NaN,NaN,1.295750e+06,4.000000,3.000000,1826.750000,NaN


In [9]:
# Since there are 10 different list types and we only want house list type
# Select rows that have list type as 'House for sale'
df_house = df[df['list_type'] == 'House for sale']
df_house

,address,locality,price,beds,bath,area_sqft,list_type
0,1728 Peregrino Way,"San Jose, CA 95125",2998000.0,5.0,4.0,3737.0,House for sale
1,3157 Tristian Ave,"San Jose, CA 95127",799000.0,3.0,2.0,1115.0,House for sale
2,593 Ezie St,"San Jose, CA 95111",749000.0,4.0,2.0,1113.0,House for sale
3,4610 Capitola Ave,"San Jose, CA 95111",848000.0,3.0,2.0,1170.0,House for sale
4,1774 Wilcox Way,"San Jose, CA 95125",2138000.0,4.0,3.0,2370.0,House for sale
...,...,...,...,...,...,...,...
740,2717 Florence Ave,"San Jose, CA 95127",699000.0,3.0,1.0,816.0,House for sale
744,899 N 5th St,"San Jose, CA 95112",899000.0,2.0,1.0,864.0,House for sale
766,1193 S 9th St,"San Jose, CA 95112",899000.0,2.0,2.0,1040.0,House for sale
778,211 Castillon Way,"San Jose, CA 95119",1350000.0,4.0,3.0,2324.0,House for sale


In [10]:
# When checking unique value for locality, we find some houses that located outside of the San Jose
df_house['locality'].unique()

array(['San Jose, CA 95125', 'San Jose, CA 95127', 'San Jose, CA 95111',
       'San Jose, CA 95136', 'San Jose, CA 95124', 'San Jose, CA 95112',
       'San Jose, CA 95148', 'San Jose, CA 95128', 'San Jose, CA 95138',
       'San Jose, CA 95122', 'Campbell, CA 95008', 'San Jose, CA 95131',
       'San Jose, CA 95117', 'San Jose, CA 95121', 'San Jose, CA 95126',
       'San Jose, CA 95118', 'San Jose, CA 95120', 'San Jose, CA 95110',
       'San Jose, CA 95129', 'San Jose, CA 95123', 'San Jose, CA 95116',
       'San Jose, CA 95135', 'San Jose, CA 95132', 'Del Cabo, CA 23406',
       'San Jose, CA 95119', 'San Jose, CA 95130', 'San Jose, CA 95133',
       'San Jose, CA 95002', 'San Jose, CA 95139', 'Alviso, CA 95134'],
      dtype=object)

In [11]:
# Select houses that are inside the city of San Jose
df_house_sj = df_house[df_house['locality'].str.contains('san jose', case=False)]
df_house_sj

,address,locality,price,beds,bath,area_sqft,list_type
0,1728 Peregrino Way,"San Jose, CA 95125",2998000.0,5.0,4.0,3737.0,House for sale
1,3157 Tristian Ave,"San Jose, CA 95127",799000.0,3.0,2.0,1115.0,House for sale
2,593 Ezie St,"San Jose, CA 95111",749000.0,4.0,2.0,1113.0,House for sale
3,4610 Capitola Ave,"San Jose, CA 95111",848000.0,3.0,2.0,1170.0,House for sale
4,1774 Wilcox Way,"San Jose, CA 95125",2138000.0,4.0,3.0,2370.0,House for sale
...,...,...,...,...,...,...,...
740,2717 Florence Ave,"San Jose, CA 95127",699000.0,3.0,1.0,816.0,House for sale
744,899 N 5th St,"San Jose, CA 95112",899000.0,2.0,1.0,864.0,House for sale
766,1193 S 9th St,"San Jose, CA 95112",899000.0,2.0,2.0,1040.0,House for sale
778,211 Castillon Way,"San Jose, CA 95119",1350000.0,4.0,3.0,2324.0,House for sale


In [12]:
# remove duplicate address
df_house_sj1 = df_house_sj.drop_duplicates(subset =['address'])

In [13]:
# now we a total of 250 unique house address and 1 list_type that is 'House for sale'
df_house_sj1.describe(include='all')

,address,locality,price,beds,bath,area_sqft,list_type
count,262,262,2.620000e+02,262.000000,262.000000,262.000000,262
unique,262,27,NaN,NaN,NaN,NaN,1
top,6228 Balderstone Dr,"San Jose, CA 95125",NaN,NaN,NaN,NaN,House for sale
freq,1,46,NaN,NaN,NaN,NaN,262
mean,NaN,NaN,1.418309e+06,3.664122,2.625954,1908.904580,NaN
std,NaN,NaN,9.639926e+05,1.025429,1.150285,973.485007,NaN
min,NaN,NaN,2.549000e+05,1.000000,1.000000,668.000000,NaN
25%,NaN,NaN,9.882500e+05,3.000000,2.000000,1297.250000,NaN
50%,NaN,NaN,1.236500e+06,4.000000,2.000000,1646.000000,NaN
75%,NaN,NaN,1.537485e+06,4.000000,3.000000,2266.750000,NaN


In [14]:
# let's look for null value
df_house_sj1.isna().sum()
# There are 1 null for beds and bath 

address      0
locality     0
price        0
beds         0
bath         0
area_sqft    0
list_type    0
dtype: int64

In [15]:
# looking for the row with na value
df_house_sj1[df_house_sj1['beds'].isna()]

,address,locality,price,beds,bath,area_sqft,list_type


In [16]:
# dropping the row with null value
df_house_sj = df_house_sj1.dropna()
df_house_sj1.isna().sum()

address      0
locality     0
price        0
beds         0
bath         0
area_sqft    0
list_type    0
dtype: int64

In [38]:
# sort the data by locality and price
df_house_sj = df_house_sj.sort_values(by=['locality','price']) 
df_house_sj

,address,locality,price,beds,bath,area_sqft,list_type
338,1355 Michigan Ave,"San Jose, CA 95002",1399000.0,5.0,5.0,2690.0,House for sale
31,1223 Mastic St,"San Jose, CA 95110",715000.0,3.0,2.0,1058.0,House for sale
116,759 Palm St,"San Jose, CA 95110",899000.0,3.0,2.0,1374.0,House for sale
730,116 Fox Ave,"San Jose, CA 95110",1400000.0,4.0,3.0,1218.0,House for sale
2,593 Ezie St,"San Jose, CA 95111",749000.0,4.0,2.0,1113.0,House for sale
...,...,...,...,...,...,...,...
95,2387 Pentland Way,"San Jose, CA 95148",1299000.0,5.0,3.0,2076.0,House for sale
172,3176 Whitesand Dr,"San Jose, CA 95148",1299888.0,5.0,4.0,2251.0,House for sale
332,2622 Raritan Pl,"San Jose, CA 95148",1350000.0,3.0,3.0,1966.0,House for sale
14,3234 Remington Way,"San Jose, CA 95148",1399800.0,4.0,2.0,2119.0,House for sale


In [22]:
# export our data to an excel sheet
df_house_sj.to_excel("BayArea_House.xlsx", sheet_name='SJ', index=False)